In [1]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['backend'] = "Qt4Agg"

client = MongoClient()
db = client.primer
coll = db.dataset

#Simple Foods
empty = np.array([0.,0.,0.,0.])
Egg = np.array([78., 5., 0.6, 6.])
Avocado_whole = np.array([234., 21., 12., 2.9])
Sour_Cream_1tbsp = np.array([23., 2.4, 0.3, 0.2])
Spinach = np.array([7., 0.1, 1.1, 0.9])
Chicken_1oz = np.array([32.5, 0.75, 0, 6.5])
Peanut_Butter_2tbsp = np.array([200., 16., 6., 8.])
Refried_Beans_1serving = np.array([195, 0.89, 33.7, 10.64])
Trail_Mix_hfcup = np.array([360, 24., 31.2, 12.])
Protein_Powder_1scoop = np.array([140., 2., 6., 24.])
Oatmeal_hfcup = np.array([150.0, 3.0, 27.0, 5.0])
Sweet_Potato_1oz = np.array([86./3.5274, 0.1/3.5274, 20./3.5274, 1.6/3.5274])
White_Rice_1cp = np.array([206, 0.4, 45.0, 4.3])
One_Percent_Milk_1cup = np.array([130, 2.5, 16., 11.])
Cottage_Cheese_hfcup = np.array([80.0, 0.0, 6.0, 13.0])

#Protein
Amplified_Mass_1scoop = np.array([187.5, 1.5, 31, 12.5])
Protein_Powder_MochaCappuccino = np.array([120, 1.5, 3., 24.])
Protein_Powder_DoubleChocolate = np.array([120, 1., 3., 24.])

#Mealy Foods
chicken_serving_small = 8*Chicken_1oz
chicken_serving_large = 10*Chicken_1oz
halfAnAvocado = np.multiply(0.5, Avocado_whole)
quarterAnAvocado = np.multiply(0.25, Avocado_whole)
omelette = 6*Egg + halfAnAvocado + 6*Sour_Cream_1tbsp + Spinach
omeletteLessFat = 6*Egg + quarterAnAvocado + Spinach
oatmeal = 2*Oatmeal_hfcup
halfMass = 2*Amplified_Mass_1scoop
halfMassWithMilk = halfMass + One_Percent_Milk_1cup

In [3]:
weight = 157.8
bodyfatpercentage = 12
sleep = {"hours": 9, "minutes": 53}

firstMeal = Protein_Powder_DoubleChocolate + omeletteLessFat
print("First Meal Calories: " + `firstMeal[0]`)
print("First Meal Fat: " + `firstMeal[1]`)
print("First Meal Carbs: " + `firstMeal[2]`)
print("First Meal Protein: " + `firstMeal[3]` + "\n")

secondMeal = 3*oatmeal
print("Second Meal Calories: " + `secondMeal[0]`)
print("Second Meal Fat: " + `secondMeal[1]`)
print("Second Meal Carbs: " + `secondMeal[2]`)
print("Second Meal Protein: " + `secondMeal[3]` + "\n")

thirdMeal = Protein_Powder_DoubleChocolate + 8.8*Chicken_1oz
print("Third Meal Calories: " + `thirdMeal[0]`)
print("Third Meal Fat: " + `thirdMeal[1]`)
print("Third Meal Carbs: " + `thirdMeal[2]`)
print("Third Meal Protein: " + `thirdMeal[3]` + "\n")

fourthMeal = empty
print("Fourth Meal Calories: " + `fourthMeal[0]`)
print("Fourth Meal Fat: " + `fourthMeal[1]`)
print("Fourth Meal Carbs: " + `fourthMeal[2]`)
print("Fourth Meal Protein: " + `fourthMeal[3]` + "\n")

total = firstMeal + secondMeal + thirdMeal + fourthMeal
print("Total Calories: " + `total[0]`)
print("Total Fat: " + `total[1]`)
print("Total Carbs: " + `total[2]`)
print("Total Protein: " + `total[3]`)


First Meal Calories: 653.5
First Meal Fat: 36.350000000000001
First Meal Carbs: 10.699999999999999
First Meal Protein: 61.625

Second Meal Calories: 900.0
Second Meal Fat: 18.0
Second Meal Carbs: 162.0
Second Meal Protein: 30.0

Third Meal Calories: 406.0
Third Meal Fat: 7.6000000000000005
Third Meal Carbs: 3.0
Third Meal Protein: 81.200000000000003

Fourth Meal Calories: 0.0
Fourth Meal Fat: 0.0
Fourth Meal Carbs: 0.0
Fourth Meal Protein: 0.0

Total Calories: 1959.5
Total Fat: 61.950000000000003
Total Carbs: 175.69999999999999
Total Protein: 172.82499999999999


In [ ]:
import matplotlib.pyplot as plt



plt.savefig(`date.month` + "-" + `date.day` + "-" + `date.year` + ".png")


In [32]:
daysShown = 7
earliestDate = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - daysShown
cursor = db.macros.find( { 'date': { '$gt': earliestDate } } )

days = []
calories = []
fat = []
carbs = []
protein = []
sleep = []
weight = []

for document in cursor:
    days.append(document['date'])
    calories.append(document['macros']['calories'])
    fat.append(document['macros']['fat'])
    carbs.append(document['macros']['carbohydrates'])
    protein.append(document['macros']['protein'])
    weight.append(document['weight'])
    sleep.append((document['sleep']['hours'],document['sleep']['minutes']))
    print(document)


{u'weight': 158.8, u'macros': {u'carbohydrates': 253.0, u'calories': 2250.0, u'protein': 200.0, u'fat': 53.0}, u'sleep': {u'hours': 7, u'minutes': 24}, u'bodyfat%': 10, u'date': 16847, u'_id': ObjectId('56c2dc26bfdaef0adb0051f0')}
{u'weight': 159.0, u'macros': {u'carbohydrates': 272.7, u'calories': 2743.5, u'protein': 257.625, u'fat': 65.35}, u'sleep': {u'hours': 7, u'minutes': 39}, u'bodyfat%': 12, u'date': 16847, u'_id': ObjectId('56c4125abfdaefd99f82822c')}
{u'weight': 159.0, u'macros': {u'carbohydrates': 325.3853206327607, u'calories': 2551.146878720871, u'protein': 174.39982565062087, u'fat': 62.4984266031638}, u'sleep': {u'hours': 8, u'minutes': 25}, u'bodyfat%': 12, u'date': 16848, u'_id': ObjectId('56c570b5a93ba30bd6df3cd3')}
{u'weight': 159.0, u'macros': {u'carbohydrates': 276.94658388614846, u'calories': 2749.2103107104385, u'protein': 252.63472671089187, u'fat': 70.29123291943074}, u'sleep': {u'hours': 9, u'minutes': 3}, u'bodyfat%': 12, u'date': 16849, u'_id': ObjectId('56c

In [33]:
today = dt.date.today()


f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.plot(days, calories, color='y')
ax2.plot(days, weight, color='b')
ax3.plot(days, carbs, color='r')
ax4.plot(days, protein, color='g')

ax1.set_ylim([min(calories), max(calories)*1.5])
ax2.set_ylim([min(weight), max(weight)*1.1])
ax3.set_ylim([min(carbs), max(carbs)*1.5])
ax4.set_ylim([min(protein), max(protein)*1.3])

ax1.set_title('Calories')
ax2.set_title('Weight')
ax3.set_title('Carbohydrates')
ax4.set_title('Protein')


plt.savefig(`today.month` + "-" + `today.day` + "-" + `today.year` + ".png")


In [29]:
print days
print calories
print fat
print carbs
print protein
print sleep
print weight

[16847, 16847, 16848, 16849, 16850]
[2250.0, 2743.5, 2551.146878720871, 2749.2103107104385, 2719.0912717582355]
[53.0, 65.35, 62.4984266031638, 70.29123291943074, 65.48698985088166]
[253.0, 272.7, 325.3853206327607, 276.94658388614846, 384.03797017633383]
[200.0, 257.625, 174.39982565062087, 252.63472671089187, 164.75983761410671]
[(7, 24), (7, 39), (8, 25), (9, 3), (8, 5)]
[158.8, 159.0, 159.0, 159.0, 158.8]


In [30]:
daysSinceEpoch = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - 1

result = db.macros.insert_one(
    {
        "date": daysSinceEpoch,
        "macros": {
            "calories": total[0],
            "fat": total[1],
            "carbohydrates": total[2],
            "protein": total[3]
        },
        "weight": weight,
        "bodyfat%": bodyfatpercentage,
        "sleep": sleep
    }
)

NameError: name 'total' is not defined